<a href="https://colab.research.google.com/github/mahisara10/OpenSource-Sathyabama/blob/master/Sportsrecommend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Config, GPT2LMHeadModel, PreTrainedTokenizerFast

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
tokenizer = PreTrainedTokenizerFast(tokenizer_file="news.json")
tokenizer.pad_token = "[PAD]"

In [11]:
class SPORTSDataset(Dataset):
    def __init__(self, file_path, tokenizer, max_length=64):
        with open(file_path, "r", encoding="utf-8") as f:
            self.lines = [line.strip() for line in f if len(line.strip()) > 10]
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.lines)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.lines[idx],
            truncation=True,
            max_length=self.max_length,
            padding="max_length",
            return_tensors="pt"
        )
        # For LLMs, 'labels' are just the input_ids shifted by one
        encoding['labels'] = encoding['input_ids'].clone()
        return {k: v.squeeze() for k, v in encoding.items()}

In [13]:
dataset = SPORTSDataset("sports_dataset.txt", tokenizer)
loader = DataLoader(dataset, batch_size=8, shuffle=True)

config = GPT2Config(vocab_size=5000, n_positions=128, n_embd=256, n_layer=4, n_head=4)
model = GPT2LMHeadModel(config)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4)



In [ ]:
model.train()
print("Starting Sports LLM Training...")

for epoch in range(3): # Run through the data 3 times
    total_loss = 0
    for batch in loader:
        optimizer.zero_grad()
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward() # This is where the model "learns" from its mistakes
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1} Complete. Average Loss: {total_loss/len(loader):.4f}")


Starting Sports LLM Training...


`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Epoch 1 Complete. Average Loss: 4.5771
